# Laboratoire 4 : Développement d'un système intelligent
#### Département du génie logiciel et des technologies de l’information

| Étudiants             |                                                         |
|-----------------------|---------------------------------------------------------|
| Jean-Philippe Decoste |  DECJ19059105                                           |
| Ahmad Al-Taher        |   ALTA22109307                                          |
| Stéphanie Lacerte     |   LACS06629109                                          |
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2018                                            |
| Groupe                | 2                                                       |
| Numéro du laboratoire | 04                                                      |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 13 dev 2018                                             |

In [1]:
import csv
import math
import os

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import decomposition
import tensorflow as tf
from tensorflow import keras
from tabulate import tabulate
import matplotlib.pyplot as plt

from helpers import utilities as Utils
from helpers import datasets as Data
from algos import Knn

## Paramètres

In [2]:
#MERGED_GALAXY_PRIMITIVE = r"data\csv\eq07_pMerged.csv"
TAGGED_PRIMITIVE = r"music\tagged_feature_sets\msd-jmirmfccs_dev\msd-jmirmfccs_dev.csv"
UNTAGGED_PRIMITIVE = r"music\untagged_feature_sets\msd-jmirmfccs_test_nolabels\msd-jmirmfccs_test_nolabels.csv"
TAGGED_PRIMITIVE2 = r"music\tagged_feature_sets\msd-jmirderivatives_dev\msd-jmirderivatives_dev.csv"
UNTAGGED_PRIMITIVE2 = r"music\untagged_feature_sets\msd-jmirmfccs_test_nolabels\msd-jmirmfccs_test_nolabels.csv"



allFeatures = []
allAnswers = []

features_train = []
answers_train = []
features_test = []
answers_test = []

### Main

In [ ]:

allFeatures,allAnswers = Data.prepareTrainDataset("Music", TAGGED_PRIMITIVE)

#determine best dementions to use
#pca
transformed =  decomposition.PCA(n_components=0.95).fit_transform(allFeatures)
#faster tests
#transformed =  decomposition.PCA(n_components=5).fit_transform(allFeatures)
print("%d features after transformation\n" % len(transformed[0]))
#determine best classifier and best model
features_train,answers_train,features_test,answers_test = Data.splitDataSet(transformed,allAnswers, 10, 0.2, 0)
print("features_train %d; answers_train %d; features_test %d; answers_test %d"%(len(features_train),len(answers_train),len(features_test),len(answers_test)))

knn = Knn.knn(features_train,answers_train,False)

#combine

PREPARING DATASETS
Reading Music features:
Progress |**************************************************| 100.0% Complete
-> Done

14 features after transformation

Splitting Dataset according to these params:
Property        Value
------------  -------
n_splits         10
test_size         0.2
random_state      0
-> Done


features_train 1436440; answers_train 1436440; features_test 359110; answers_test 359110
1.Training 



## Introduction et revue de la littérature

## Question 2
### Configuration de l'environnement

### Partitionnement des données

### Validation

### Prétraitement

## Question 3
### Description des modèles et justifications

## Question 4
### Matrice des expérimentations

### Matrice des résultats de l'étude des hyperparamètres

### Graphiques

### Discussion

## Question 5
### Présentation de la conception

## Question 6
### Hyperparamètres des modèles choisis dans la conception

### Matrice des résultats

### Discussion

## Question 7 
### Formulation des recommandations

## Conclusion